In [1]:
import os
import pandas as pd

from datasets import load_dataset, concatenate_datasets
from openai import AsyncOpenAI
import warnings

# すべての警告を無視
warnings.filterwarnings("ignore")

In [2]:
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")

client = AsyncOpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY,  # 取得したAPIキーを入力してください
)

### データセットの準備

In [3]:
jaquad_dataset = load_dataset("SkelterLabsInc/JaQuAD")

combined_data = concatenate_datasets(
    [jaquad_dataset["train"], jaquad_dataset["validation"]]
)

# 前処理：正解データの準備
combined_data_processed = []
for item in combined_data:
    answer_text = item["answers"]["text"][0] if item["answers"]["text"] else ""
    answer_start = (
        item["answers"]["answer_start"][0] if item["answers"]["answer_start"] else -1
    )
    combined_data_processed.append(
        {
            "question": item["question"],
            "context": item["context"],
            "answer_text": answer_text,
            "answer_start": answer_start,
        },
    )


# データが多いため10件に絞る
combined_data_processed = combined_data_processed[:10]
combined_data_processed[0]

{'question': '戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?',
 'context': '手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989年(平成元年)2月9日)は、日本の漫画家、アニメーター、アニメ監督である。\n戦後日本においてストーリー漫画の第一人者として、漫画表現の開拓者的な存在として活躍した。\n\n兵庫県宝塚市出身(出生は大阪府豊能郡豊中町、現在の豊中市)同市名誉市民である。\n大阪帝国大学附属医学専門部を卒業。\n医師免許取得のち医学博士(奈良県立医科大学・1961年)。',
 'answer_text': '手塚治虫',
 'answer_start': 0}

### llama3-70b-instructを用いて回答作成

In [50]:
async def rag_generate_answer(client, question: str, context: str) -> str:
    """APIを使ってLLMによる解答を得る関数

    Question (str): 質問内容
        context (str): 解答の根拠になるテキスト情報

    Returns:
        response.choices[0].message.content(str): contextを元に回答したLLMの回答内容

    """
    response = await client.chat.completions.create(
        model="nvidia/llama-3.1-nemotron-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": "あなたは優秀なアシスタントです。"
                "これから渡すコンテキストを元に一問一答形式の問題に答えてください。"
                "答えられる文字数は少ないので「です、ます」をつける敬体は絶対にやめて単語のみ回答してください。",
            },
            {
                "role": "user",
                "content": f"Context: {context}. Answer the question:{question}",
            },
        ],
        max_tokens=30,
        temperature=0,
    )

    return response.choices[0].message.content


async def llm_generate_answer(client, question: str) -> str:
    """APIを使ってLLMによる解答を得る関数

    Question (str): 質問内容
        context (str): 解答の根拠になるテキスト情報

    Returns:
        response.choices[0].message.content(str): contextを元に回答したLLMの回答内容

    """
    response = await client.chat.completions.create(
        model="nvidia/llama-3.1-nemotron-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": "あなたは優秀なアシスタントです。一問一答形式の問題に答えてください。答えられる文字数は少ないので「です、ます」をつける敬体は絶対にやめて単語のみ回答してください。",
            },
            {
                "role": "user",
                "content": f"Answer the question: {question}",
            },
        ],
        max_tokens=30,
        temperature=0,
    )
    return response.choices[0].message.content

In [51]:
for item in combined_data_processed:
    item["rag_answer_text"] = await rag_generate_answer(
        client, item["question"], item["context"]
    )
    item["llm_answer_text"] = await llm_generate_answer(client, item["question"])

### nemotron-4-340b-rewardで評価

In [53]:
def get_scores_from_response(openai_response_template):
    """OpenAI APIのレスポンスからトークンごとのスコアを抽出する関数

    Args:
        openai_response_template: OpenAI APIのレスポンスオブジェクト

    Returns:
        dict: トークンとそのログ確率を含む辞書
    """
    logprobs = openai_response_template.choices[0].logprobs.content
    score_dict = {}
    for score in logprobs:
        score_dict[score.token] = score.logprob
    return score_dict


async def get_response_and_scores(client, question, response_content):
    """指定された質問と応答内容をもとにスコアリングを行う関数

    Args:
        client: OpenAI APIのクライアントオブジェクト
        question (str): 質問文
        response_content (str): 応答内容

    Returns:
        dict: トークンごとのスコアを含む辞書
    """
    messages = [
        {"role": "user", "content": question},
        {"role": "assistant", "content": response_content},
    ]
    response = await client.chat.completions.create(
        model="nvidia/nemotron-4-340b-reward",
        messages=messages,
    )
    scores = get_scores_from_response(response)
    return scores


async def rag_process_question_response_pairs(client, combined_data_processed):
    """質問と応答のペアに対してスコアリングを行う関数

    Args:
        client: OpenAI APIのクライアントオブジェクト
        combined_data_processed (list): 質問と応答のペアを含むデータリスト

    Returns:
        list: 各質問と応答ペアごとのスコア情報を含むリスト
    """
    results = []
    for i in range(len(combined_data_processed)):
        question = combined_data_processed[i]["question"]
        answer = combined_data_processed[i]["rag_answer_text"]

        scores = await get_response_and_scores(
            client,
            question,
            answer,
        )
        results.append((question, answer, scores))
    return results


async def llm_process_question_response_pairs(client, combined_data_processed):
    """質問と応答のペアに対してスコアリングを行う関数

    Args:
        client: OpenAI APIのクライアントオブジェクト
        combined_data_processed (list): 質問と応答のペアを含むデータリスト

    Returns:
        list: 各質問と応答ペアごとのスコア情報を含むリスト
    """
    results = []
    for i in range(len(combined_data_processed)):
        question = combined_data_processed[i]["question"]
        answer = combined_data_processed[i]["llm_answer_text"]

        scores = await get_response_and_scores(
            client,
            question,
            answer,
        )
        results.append((question, answer, scores))
    return results

In [54]:
llm_results = await llm_process_question_response_pairs(client, combined_data_processed)
rag_results = await rag_process_question_response_pairs(client, combined_data_processed)

In [55]:
llm_results_df = pd.DataFrame(
    [
        (
            question,
            answer,
            scores["helpfulness"],
            scores["correctness"],
            scores["coherence"],
            scores["complexity"],
            scores["verbosity"],
        )
        for question, answer, scores in llm_results
    ],
    columns=[
        "question",
        "llm_answer",
        "helpfulness",
        "correctness",
        "coherence",
        "complexity",
        "verbosity",
    ],
)
llm_results_df

,question,llm_answer,helpfulness,correctness,coherence,complexity,verbosity
0,戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?,手塚治虫（てづかおさむ）,2.828125,2.828125,3.562500,0.789062,0.734375
1,手塚治虫の出身地はどこになりますか?,大阪府豊中市,1.679688,1.546875,3.625000,0.445312,0.687500
2,手塚治虫の漫画家としてのデビュー作は何かな?,新宝島です。,2.281250,2.421875,3.718750,0.476562,0.447266
3,1947年に大阪に赤本ブームが起こったのはなぜ?,1947年に大阪で起こった赤本ブームの主な理由は以下の通りです。\n\n1. **戦後の社会...,1.085938,0.851562,2.593750,0.695312,1.070312
4,日本で一番初めに30分枠のテレビアニメとなった作品名は何?,オタスケ娘,0.478516,0.582031,3.093750,0.104004,0.593750
5,手塚治虫の代表作である『鉄腕アトム』、『リボンの騎士』、『アドルフに告ぐ』の中で、最も遅く発...,アドルフに告ぐ,3.343750,3.796875,3.828125,0.878906,0.761719
6,手塚治虫が軍事色が強かった時期にも関わらず、精力的に漫画活動を行った学校の名前は?,神戸山手学園中学校,1.867188,2.062500,3.437500,0.605469,0.808594
7,『紙の砦』や『どついたれ』などの自伝的作品の中にも描写されている手塚治虫にとって命を落として...,戦争中、手塚治虫が体験した出来事は、米軍機による機銃掃射です。,2.078125,2.531250,3.625000,1.007812,0.843750
8,『紙の砦』と『どついたれ』の作品はどちらが早く描かれたの?,『どついたれ』,0.214844,0.550781,3.078125,0.369141,0.120117
9,『マアチャンの日記帳』の連載期間はいつからいつまで?,1946年～1948年,1.484375,1.585938,3.453125,0.478516,0.425781


In [56]:
rag_results_df = pd.DataFrame(
    [
        (
            question,
            answer,
            scores["helpfulness"],
            scores["correctness"],
            scores["coherence"],
            scores["complexity"],
            scores["verbosity"],
        )
        for question, answer, scores in rag_results
    ],
    columns=[
        "question",
        "rag_answer",
        "helpfulness",
        "correctness",
        "coherence",
        "complexity",
        "verbosity",
    ],
)

In [57]:
rag_results_df["answer"] = ""
rag_results_df["context"] = ""
for i in range(len(rag_results_df)):
    rag_results_df["answer"][i] = combined_data_processed[i]["answer_text"]
    rag_results_df["context"][i] = combined_data_processed[i]["context"]

rag_results_df["llm_answer"] = llm_results_df["llm_answer"]

In [58]:
rag_results_df = rag_results_df[
    [
        "question",
        "context",
        "answer",
        "llm_answer",
        "rag_answer",
        "helpfulness",
        "correctness",
        "coherence",
        "complexity",
        "verbosity",
    ]
]
rag_results_df

,question,context,answer,llm_answer,rag_answer,helpfulness,correctness,coherence,complexity,verbosity
0,戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?,手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989...,手塚治虫,手塚治虫（てづかおさむ）,手塚治虫,3.109375,3.171875,3.687500,0.792969,0.691406
1,手塚治虫の出身地はどこになりますか?,手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989...,兵庫県宝塚市,大阪府豊中市,兵庫県宝塚市（出生は大阪府豊能郡豊中町、現在の豊中市）,3.390625,3.390625,3.812500,0.984375,0.910156
2,手塚治虫の漫画家としてのデビュー作は何かな?,大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少...,『マアチャンの日記帳』,新宝島です。,マアチャンの日記帳,2.328125,2.453125,3.390625,0.585938,0.722656
3,1947年に大阪に赤本ブームが起こったのはなぜ?,大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少...,『新寶島』がベストセラーとなり,1947年に大阪で起こった赤本ブームの主な理由は以下の通りです。\n\n1. **戦後の社会...,酒井七馬原案の描き下ろし単行本『新寶島』がベストセラーとなったからです。,1.789062,2.015625,3.390625,0.785156,0.644531
4,日本で一番初めに30分枠のテレビアニメとなった作品名は何?,大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少...,『鉄腕アトム』,オタスケ娘,鉄腕アトム,2.500000,2.906250,3.515625,0.757812,0.730469
5,手塚治虫の代表作である『鉄腕アトム』、『リボンの騎士』、『アドルフに告ぐ』の中で、最も遅く発...,大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少...,『アドルフに告ぐ』,アドルフに告ぐ,アドルフに告ぐ,3.328125,3.781250,3.828125,0.875000,0.769531
6,手塚治虫が軍事色が強かった時期にも関わらず、精力的に漫画活動を行った学校の名前は?,1941年、大阪府立北野中学校(現在の大阪府立北野高等学校)に入学した。\n時節柄軍事色が強...,大阪府立北野中学校,神戸山手学園中学校,大阪府立北野中学校（現在の大阪府立北野高等学校）,2.421875,2.578125,3.484375,0.800781,0.914062
7,『紙の砦』や『どついたれ』などの自伝的作品の中にも描写されている手塚治虫にとって命を落として...,1941年、大阪府立北野中学校(現在の大阪府立北野高等学校)に入学した。\n時節柄軍事色が強...,大阪大空襲,戦争中、手塚治虫が体験した出来事は、米軍機による機銃掃射です。,大阪大空襲,0.228516,0.384766,2.937500,0.388672,0.236328
8,『紙の砦』と『どついたれ』の作品はどちらが早く描かれたの?,1941年、大阪府立北野中学校(現在の大阪府立北野高等学校)に入学した。\n時節柄軍事色が強...,『紙の砦』,『どついたれ』,『紙の砦』(1974年)が早く描かれた。,2.609375,2.718750,3.718750,0.671875,0.585938
9,『マアチャンの日記帳』の連載期間はいつからいつまで?,終戦後、学生である手塚は戦時中に描き溜めた長編の中から『幽霊男』(『メトロポリス』の原型)と...,1946年1月1日-3月31日,1946年～1948年,1946年1月1日～3月31日,2.234375,2.375000,3.515625,0.671875,0.613281
